In [2]:
import sys
from pathlib import Path

# Specify the path to your modules directory
module_path = Path("/Users/elyestabbane/Documents/UNICOG/2-Experiments/replayseq/1-Scripts/analysis_scripts")

# Add the directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Import your modules
from modules import *
import pandas as pd
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [3]:
import os
import mne
from mne_bids import BIDSPath, write_raw_bids, write_meg_calibration, write_meg_crosstalk
import json
import numpy as np

# Path to BIDS directory
root_path="/Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data"
path_BIDS =os.path.join(root_path,"BIDS")
path_raw=os.path.join(root_path,"raw")

In [4]:
# 0. Define subject and run
sub_list=range(18)

# 0.b. Load bad channels JSON data
path_bad_channels="/Users/elyestabbane/Documents/UNICOG/2-Experiments/replayseq/1-Scripts/analysis_scripts/modules/objects/bad_channels.json"
with open(path_bad_channels, 'r') as file:
        bad_channels_dict = json.load(file)

for sub in sub_list:
    for run in range(1,19):
        # 1. load the data
        lab=bad_channels_dict[f"sub-{sub:02}"]['lab']
        original_data_path=os.path.join(path_raw,"Data_ICM" if lab=='icm' else f'Data_neurospin')
        data_path=os.path.join(original_data_path,f"sub-{sub:02}/run{run:02}_raw.fif")
        print(f"--- Subject-{sub:02} ({lab}): saving in bids format run-{run} ------")
        raw=mne.io.read_raw_fif(data_path,allow_maxshield=True,preload=True)
        
        
        # 2. Add the events
        print('Adding Events')        
        n_chans=raw.get_data().shape[0]
        raw.pick(np.where(['EEG' not in raw.info['ch_names'][i] for i in range(n_chans)])[0])
        
        if lab=='icm':
            raw.set_channel_types({'BIO001': 'eog'})
            raw.set_channel_types({'BIO002': 'eog'})
            raw.set_channel_types({'BIO003': 'ecg'})
            events, event_ids = extract_events_and_event_IDs_ICM(raw)
            
        else:
            events, event_ids = extract_events_and_event_IDs_neurospin(raw)
        bids_path = BIDSPath(subject=f'{sub:02}', task='reproduction', run=run, datatype='meg', root=path_BIDS)
        
        # 3. Add the bad channels
        raw.info['bads']=bad_channels_dict[f"sub-{sub:02}"][f"run{run:02}"]
        
        # 4. Annotate the muscle artifacts
        print('Annotating muscle artifact')
        annot_muscle, scores_muscle = mne.preprocessing.annotate_muscle_zscore(
            raw,
            ch_type="mag",
            threshold=5,
            min_length_good=0.2,
            filter_freq=[100, 300],
        )
        raw.set_annotations(annot_muscle)
        
        # 5. Save the data
        write_raw_bids(raw, bids_path=bids_path,allow_preload=True,format='FIF',events=events,event_id=event_ids,overwrite=True)
        print(f'File written in {bids_path}')
        
        # 6. write MEG calibration files
        print('writing MEG calibration files')
        if lab=='icm':
            ct_fname = path_BIDS + "/calibration_files/calibration_icm/ct_sparse.fif"
            cal_fname = path_BIDS + "/calibration_files/calibration_icm/sss_cal_3101_160108.dat"
        else:
            ct_fname = path_BIDS + "/calibration_files/calibration_neurospin/ct_sparse.fif"
            cal_fname = path_BIDS + "/calibration_files/calibration_neurospin/sss_cal_3176_20240123_2.dat"
        
        
        write_meg_calibration(calibration=cal_fname,bids_path=bids_path)
        write_meg_crosstalk(fname=ct_fname,bids_path=bids_path)

--- Subject-16 (neurospin): saving in bids format run-1 ------
Opening raw data file /Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/raw/Data_neurospin/sub-16/run01_raw.fif...
    Read a total of 13 projection items:
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
    Range : 52000 ... 468999 =     52.000 ...   468.999 secs
Ready.
Reading 0 ... 416999  =      0.000 ...   416.999 secs...


/var/folders/qn/h72tf80n225331v43m72lykw0000gn/T/ipykernel_64886/2627218118.py:16: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw=mne.io.read_raw_fif(data_path,allow_maxshield=True,preload=True)


Adding Events
Annotating muscle artifact
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1e+02 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 100.00
- Lower transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 87.50 Hz)
- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutoff frequency: 337.50 Hz)
- Filter length: 133 samples (0.133 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1651 samples (1.651 s)

Writing '/Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/BIDS/participants.tsv'...
Writing '/Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/BIDS/participants.json'...
Writing '/Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/BIDS/sub-16/meg/sub-16_coordsystem.json'...
Writing '/Volumes/T5_EVO/1-experiments/REPLAYSEQ/2-Data/BIDS/sub-16/meg/sub-16_coordsystem.json'...
Used Annotations descriptions: ['BAD_muscle', 'SequenceID-Rep4/Position-1', 'SequenceID-Rep4/Position-2', 'SequenceID-Rep4/Position-3', 'SequenceID-Rep4/Position-4', 'SequenceID-Rep4/Position-5', 'SequenceID-Rep4/Position-6', 'f

In [36]:
raw.info.keys()

dict_keys(['file_id', 'events', 'hpi_results', 'hpi_meas', 'subject_info', 'device_info', 'helium_info', 'hpi_subsystem', 'proc_history', 'meas_id', 'experimenter', 'description', 'proj_id', 'proj_name', 'meas_date', 'utc_offset', 'sfreq', 'highpass', 'lowpass', 'line_freq', 'gantry_angle', 'chs', 'dev_head_t', 'ctf_head_t', 'dev_ctf_t', 'dig', 'bads', 'ch_names', 'nchan', 'projs', 'comps', 'acq_pars', 'acq_stim', 'custom_ref_applied', 'xplotter_layout', 'kit_system_id', 'maxshield'])